In [1]:
!git clone https://github.com/ultralytics/yolov3  # clone repo
%cd yolov3
%pip install -qr requirements.txt  # install dependencies

fatal: destination path 'yolov3' already exists and is not an empty directory.
/home/dimitri/projects/ifishienci/yolov3
Note: you may need to restart the kernel to use updated packages.


In [2]:
model_weights = "/home/dimitri/projects/ML_playground/yolov3/runs/train/exp15/weights/best.pt"
path = "/home/dimitri/projects/ifishienci/data/fish" #indicate where the files are located
data_location = 'data/fish/' #end of path 

In [3]:
import torch
from IPython.display import Image, clear_output  # to display images
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
import numpy as np

import numpy as np
import pandas as pd
import pickle
import ast

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score


clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.1 _CudaDeviceProperties(name='GeForce GTX 1650', major=7, minor=5, total_memory=3914MB, multi_processor_count=16)


In [4]:
opt ={"agnostic_nms":False, "augment":False, "classes":None, "conf_thres":0.25, "device":'', "exist_ok":False, "img_size":640, "iou_thres":0.45, "name":'exp', "project":'runs/detect', "save_conf":False, "save_txt":True, "source":'data/images/fish', "update":False, "view_img":False, "weights":model_weights}


In [5]:
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, non_max_suppression, apply_classifier, scale_coords, \
    xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

def detect(opt):
    coord = []
    source, weights, view_img, save_txt, imgsz = opt['source'], opt["weights"], opt["view_img"], opt["save_txt"], opt["img_size"]
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://'))

    # Directories
    save_dir = Path(increment_path(Path(opt["project"]) / opt["name"], exist_ok=opt["exist_ok"]))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(opt["device"])
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages(source, img_size=imgsz)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=opt['augment'])[0]


        # Apply NMS
        pred = non_max_suppression(pred, opt["conf_thres"], opt['iou_thres'], classes=opt["classes"], agnostic=opt["agnostic_nms"])
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], '%g: ' % i, im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f'{n} {names[int(c)]}s, '  # add to string

                
                # Write results
                normal_coord = []
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        normal_coord.append(xywh)

                        line = (cls, *xywh, conf) if opt["save_conf"] else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or view_img:  # Add bbox to image
                        label = f'{names[int(cls)]} {conf:.2f}'
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

                    

            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            
            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fourcc = 'mp4v'  # output video codec
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)

        bbox = det[:,:4]
        conf = det[:,4]
        class_id = det[:,5]

        x = bbox[0:,0].cpu().data.numpy()
        y = bbox[0:,1].cpu().data.numpy()
        
        for xi, yi in zip(x,y):
            coord.append([xi,yi])
        plt.plot(x, y, 'o', color='black');
        result = str(source).split('/')[-1]
        plt.savefig("/home/dimitri/projects/ML_playground/yolov3/yolov3/data/splitted_fish/after_results/"+str(result)+'.png')
    plt.clf()
    return bbox, conf, class_id, coord
    
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

In [6]:
import os

directories = os.listdir(path)

In [7]:
directories

['2', '1']

In [9]:
global_coord = {}
for i in directories:
    opt['source'] = data_location+str(i)
    bbox, conf, class_id, coord = detect(opt)
    global_coord[str(i)] = coord

YOLOv3 🚀 v9.5.0-11-gab7ff9d torch 1.7.1 CUDA:0 (GeForce GTX 1650, 3914.1875MB)

Fusing layers... 
Model Summary: 261 layers, 61497430 parameters, 0 gradients


image 1/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_21.png: 384x640 16 fishs, Done. (0.180s)
image 2/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_22.png: 384x640 19 fishs, Done. (0.179s)
image 3/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_23.png: 384x640 19 fishs, Done. (0.173s)
image 4/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_24.png: 384x640 20 fishs, Done. (0.173s)
image 5/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_25.png: 384x640 16 fishs, Done. (0.174s)
image 6/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_26.png: 384x640 17 fishs, Done. (0.173s)
image 7/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_27.png: 384x640 15 fishs, Done. (0.173s)
image 8/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_28.png: 384x640 19 fishs, Done. (0.174s)
image 9/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/2/d1_1_29.png: 384x640 22 fishs, Done. (0.173s)
image 10/2

YOLOv3 🚀 v9.5.0-11-gab7ff9d torch 1.7.1 CUDA:0 (GeForce GTX 1650, 3914.1875MB)



384x640 21 fishs, Done. (0.173s)


Fusing layers... 
Model Summary: 261 layers, 61497430 parameters, 0 gradients


image 1/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_1.png: 384x640 17 fishs, Done. (0.167s)
image 2/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_10.png: 384x640 20 fishs, Done. (0.172s)
image 3/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_11.png: 384x640 20 fishs, Done. (0.172s)
image 4/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_12.png: 384x640 19 fishs, Done. (0.168s)
image 5/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_13.png: 384x640 20 fishs, Done. (0.168s)
image 6/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_14.png: 384x640 18 fishs, Done. (0.167s)
image 7/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_15.png: 384x640 23 fishs, Done. (0.168s)
image 8/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_16.png: 384x640 18 fishs, Done. (0.168s)
image 9/20 /home/dimitri/projects/ifishienci/yolov3/data/fish/1/d1_1_17.png: 384x640 16 fishs, Done. (0.167s)
image 10/20

In [10]:
import pickle

with open('./global_coord_after.pickle', 'wb') as handle:
    pickle.dump(global_coord, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Compute Metrics 

In [11]:
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
        distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
        return np.mean(distances)

In [12]:
global_result = {}
for lab_id, coord in global_coord.items():

    X = np.array(coord)
    kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

    labels = np.array(kmeans.fit_predict(X))
    u_labels = np.unique(labels)

    #plotting the results:
    #for i in u_labels:
        #plt.scatter(X[labels == i , 0] , X[labels == i , 1] )#, labels = i)
    #plt.show()
    centroids = kmeans.cluster_centers_

    c_mean_distances = []
    for i, (cx, cy) in enumerate(centroids):
        mean_distance = k_mean_distance(X, cx, cy, i, labels)
        c_mean_distances.append(mean_distance)

    c_mean_distances_0 = c_mean_distances[0]
    c_mean_distances_1 = c_mean_distances[1]
    c_mean_distances_2 = c_mean_distances[2]
    c_mean_distances_3 = c_mean_distances[3]
    
    ch_score = calinski_harabasz_score(X, labels)
    db_score = davies_bouldin_score(X, labels)
    s_score = silhouette_score(X, labels) 
    global_result[lab_id] = {"ch_score" : ch_score, "db_score" : db_score, "s_score" : s_score, "c_md_0": c_mean_distances_0, "c_md_1" : c_mean_distances_1,
                    "c_md_2" : c_mean_distances_2, "c_md_3" : c_mean_distances_3}

In [13]:
df = pd.DataFrame(global_result).T
df = df.sort_index()

In [20]:
df

,ch_score,db_score,s_score,c_md_0,c_md_1,c_md_2,c_md_3
1,557.140502,0.697772,0.517984,173.351036,217.737697,231.815454,174.311016
2,603.715550,0.618185,0.533260,178.778940,207.594260,154.001811,178.372759


In [21]:
df.to_csv('data_conso.csv')

In [23]:
for i in directories:
    test = global_coord[i]
    coord = np.array(test)
    X = coord
    kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

    labels = np.array(kmeans.fit_predict(X))
    u_labels = np.unique(labels)

    #plotting the results:

    for i in u_labels:
        plt.scatter(X[labels == i , 0] , X[labels == i , 1] )#, labels = i)

    plt.show()
    plt.savefig('/home/dimitri/projects/ML_playground/yolov3/yolov3/data/splitted_fish/before_restults/test.png')

<ipython-input-23-20325202d234>:15: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-23-20325202d234>:15: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
df